In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import random

import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import shutil
from zipfile import ZipFile

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam


In [ ]:
# Extract dataset
!unzip Corrosion.zip


Archive:  Corrosion.zip
 extracting: Corrosion/no rust/001_0fwtaowy.t1o.jpg  
 extracting: Corrosion/no rust/001_kwmsqgk2.yfv.jpg  
 extracting: Corrosion/no rust/002_2bvturws.zss.jpg  
 extracting: Corrosion/no rust/002_dr3selb3.a1h.jpg  
 extracting: Corrosion/no rust/002_mjcgc4b1.ru0.jpg  
 extracting: Corrosion/no rust/002_ocpp5ini.zep.jpg  
 extracting: Corrosion/no rust/002_vzt4zfyf.2j3.jpg  
 extracting: Corrosion/no rust/002_yoamnba2.wdq.jpg  
 extracting: Corrosion/no rust/002_zar1fupp.kf2.jpg  
 extracting: Corrosion/no rust/003_zomak3wq.43c.jpg  
 extracting: Corrosion/no rust/006_2vq3zs35.40h.jpg  
 extracting: Corrosion/no rust/006_esktmdp0.le4.jpg  
 extracting: Corrosion/no rust/006_pwvc1uiv.zhi.jpg  
 extracting: Corrosion/no rust/006_ri0wotdb.hsh.jpg  
 extracting: Corrosion/no rust/007_drdnswnb.xeb.jpg  
 extracting: Corrosion/rust/001_w1nojjln.bdv.jpg  
 extracting: Corrosion/rust/002_cql5orz2.p0y.jpg  
 extracting: Corrosion/rust/002_q40jac40.sd4.jpg  
 extracting: 

In [ ]:
rust_dir = '/content/Corrosion/rust'
no_rust_dir = '/content/Corrosion/no rust'
test_dir = 'Test'


In [ ]:
# Create test directories if they don't exist
os.makedirs(f'{test_dir}/rust', exist_ok=True)
os.makedirs(f'{test_dir}/no rust', exist_ok=True)

In [ ]:
# Get the list of files from the rust and no rust directories
rust_files = os.listdir(rust_dir)
no_rust_files = os.listdir(no_rust_dir)

# Randomly select 10 files from each directory
selected_rust_files = random.sample(rust_files, 10)
selected_no_rust_files = random.sample(no_rust_files, 10)

# Move selected rust files to the test set
for file in selected_rust_files:
    shutil.move(os.path.join(rust_dir, file), os.path.join(f'{test_dir}/rust', file))

# Move selected no rust files to the test set
for file in selected_no_rust_files:
    shutil.move(os.path.join(no_rust_dir, file), os.path.join(f'{test_dir}/no rust', file))

print("Test set created with 10 'rust' images and 10 'no rust' images.")

Test set created with 10 'rust' images and 10 'no rust' images.


In [ ]:
# Constant
train_dir = '/content/Corrosion'

BATCH_SIZE = 32
IMG_WIDTH, IMG_HEIGHT = 128, 128
EPOCHS = 10
NUM_CLASSES = 2  # rust and no rust

train_data = ImageDataGenerator(rescale=1./255)
test_data = ImageDataGenerator(rescale=1./255)

train_generator = train_data.flow_from_directory(
            train_dir,
            target_size=(IMG_WIDTH, IMG_HEIGHT),
            batch_size=BATCH_SIZE,
            class_mode='binary',
            color_mode='rgb' )

test_generator = test_data.flow_from_directory(
            test_dir,
            target_size=(IMG_WIDTH, IMG_HEIGHT),
            batch_size=BATCH_SIZE,
            class_mode='binary',
            color_mode='rgb',
            shuffle=False)

Found 9 images belonging to 2 classes.
Found 20 images belonging to 2 classes.


**TASK 1: DEVELOP SIMPLE CNN AND RESNET50**



1.   Simple CNN



In [ ]:
model = Sequential([
    Input(shape=(IMG_WIDTH, IMG_HEIGHT, 3)),  # Input shape for RGB images (3 channels)
    Conv2D(8, 3, activation='relu'),
    MaxPooling2D(pool_size=2),
    Flatten(),
    Dense(10, activation='relu'),
    Dense(1, activation='sigmoid'),  # Output layer for binary classification
])


In [ ]:
# Compile the model
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=test_generator
)

Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
/usr/local/lib/python3.11/dist-packages/keras/src/losses/losses.py:33: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return self.fn(y_true, y_pred, **self._fn_kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 0.4444 - loss: 0.0000e+00 - val_accuracy: 0.5000 - val_loss: 0.0000e+00
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.5556 - loss: 0.0000e+00 - val_accuracy: 0.5000 - val_loss: 0.0000e+00
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 484ms/step - accuracy: 0.5556 - loss: 0.0000e+00 - val_accuracy: 0.5000 - val_loss: 0.0000e+00
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 369ms/step - accuracy: 0.5556 - loss: 0.0000e+00 - val_accuracy: 0.5000 - val_loss: 0.0000e+00
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 387ms/step - accuracy: 0.5556 - loss: 0.0000e+00 - val_accuracy: 0.5000 - val_loss: 0.0000e+00
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 593ms/step - accuracy: 0.5556 - loss: 0.0000e+00 - val_accuracy: 0.5000 - val_loss: 0.0000e+00
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 244ms/step - accuracy: 0.5556 - loss: 0.0000e+00 - val_accuracy: 0.5000 - val_loss: 0.0000e+00
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 261ms/step - accuracy: 0.5556 - loss: 0.

In [ ]:
# Save the model weights
model.save_weights('cnn.weights.h5')

In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(test_generator)
print(f'Test accuracy: {accuracy * 100:.2f}%')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 180ms/step - accuracy: 0.5000 - loss: 0.0000e+00
Test accuracy: 50.00%


In [ ]:
# Test Set prediction
model_output = model.predict(test_generator)
predicted_classes = np.round(model_output).astype(int)  # Round model_output to get 0 or 1

true_classes = test_generator.classes

class_labels = list(test_generator.class_indices.keys())

# Test Set prediction
model_output = model.predict(test_generator)
predicted_classes = np.round(model_output).astype(int)  # Round model_output to get 0 or 1

# True labels
true_classes = test_generator.classes

# Class labels
class_labels = list(test_generator.class_indices.keys())

# Create a table of true class and predicted class
comparison_df = pd.DataFrame({
    'Filename': test_generator.filenames,
    'True Class': [class_labels[int(i)] for i in true_classes],
    'Predicted Class': [class_labels[int(i)] for i in predicted_classes]
})

#Get the result
print(comparison_df)




1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 214ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 182ms/step
                        Filename True Class Predicted Class
0   no rust/001_kwmsqgk2.yfv.jpg    no rust         no rust
1   no rust/002_2bvturws.zss.jpg    no rust         no rust
2   no rust/002_dr3selb3.a1h.jpg    no rust         no rust
3   no rust/002_mjcgc4b1.ru0.jpg    no rust         no rust
4   no rust/002_yoamnba2.wdq.jpg    no rust         no rust
5   no rust/002_zar1fupp.kf2.jpg    no rust         no rust
6   no rust/003_zomak3wq.43c.jpg    no rust         no rust
7   no rust/006_2vq3zs35.40h.jpg    no rust         no rust
8   no rust/006_esktmdp0.le4.jpg    no rust         no rust
9   no rust/006_pwvc1uiv.zhi.jpg    no rust         no rust
10     rust/002_cql5orz2.p0y.jpg       rust         no rust
11     rust/002_q40jac40.sd4.jpg       rust         no rust
12     rust/003_glbjresp.01k.jpg       rust         no rust
13     rust/004_1xgwoahx.lqq.jpg       rust         no rust
14     rust/004_i1dmci

<ipython-input-13-666d6b4a4df6>:23: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  'Predicted Class': [class_labels[int(i)] for i in predicted_classes]


In [ ]:
# Calculate final accuracy
correct_predictions = np.sum(predicted_classes.flatten() == true_classes)
final_accuracy = correct_predictions / len(true_classes)
print(f'Final Overall Accuracy: {final_accuracy * 100:.2f}%')

Final Overall Accuracy: 50.00%


In [ ]:
cnn_dir = 'cnn_test'

os.makedirs(cnn_dir, exist_ok=True) # CNN test directory


In [ ]:
# Save images with predicted labels
for i, filename in enumerate(test_generator.filenames):
    img_path = os.path.join(test_dir, filename)
    imgBGR = cv2.imread(img_path)  # Read the image in BGR format
    imgRGB = cv2.cvtColor(imgBGR, cv2.COLOR_BGR2RGB)  # Convert to RGB format

    # Get the true and predicted class labels
    true_label = class_labels[true_classes[i]]
    predicted_label = class_labels[predicted_classes[i][0]]

    # Add predicted class to the image
    imgRGB = cv2.putText(
        imgRGB, f"True: {true_label} | Predicted: {predicted_label}", (10, 30),
        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2, cv2.LINE_AA
    )

    # Save the output image
    cnn_img_path = os.path.join(cnn_dir, f"test_image_{i+1}.jpg")
    cv2.imwrite(cnn_img_path, cv2.cvtColor(imgRGB, cv2.COLOR_RGB2BGR))

2.   RESNET 50 MODEL



In [ ]:
# Constant
BATCH_SIZE = 32
IMG_SIZE = 224
NUM_CLASSES = 2

NUM_EPOCHS = 10
EARLY_STOP_PATIENCE = 3
BATCH_SIZE_TRAINING = 32
BATCH_SIZE_VALIDATION = 32



train_dir = '/content/Corrosion'


In [ ]:
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input

In [ ]:
data_generator = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = data_generator.flow_from_directory(
    train_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE_TRAINING,
    class_mode='categorical'
)

validation_generator = data_generator.flow_from_directory(
    test_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE_VALIDATION,
    class_mode='categorical'
)



Found 9 images belonging to 2 classes.
Found 20 images belonging to 2 classes.


In [ ]:
# Resnet50 model
base_model = ResNet50(include_top=False, weights='imagenet',
                      input_tensor=Input(shape=(IMG_SIZE, IMG_SIZE, 3)))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(NUM_CLASSES, activation = 'softmax')(x)
model = Model(inputs = base_model.input, outputs=x)

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
# Freeze the base model layers to prevent training them
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1             │ (None, 224, 224, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_pad (ZeroPadding2D) │ (None, 230, 230, 3)    │              0 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_conv (Conv2D)       │ (None, 112, 112, 64)   │          9,472 │ conv1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_bn                  │ (None, 112, 112, 64)   │            256 │ conv1_conv[0][0]       │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_relu (Activation)   │ (None, 112, 112, 64)   │              0 │ conv1_bn[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pad (ZeroPadding2D) │ (None, 114, 114, 64)   │              0 │ conv1_relu[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pool (MaxPooling2D) │ (None, 56, 56, 64)     │              0 │ pool1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_conv       │ (None, 56, 56, 64)     │          4,160 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_bn         │ (None, 56, 56, 64)     │            256 │ conv2_block1_1_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_relu       │ (None, 56, 56, 64)     │              0 │ conv2_block1_1_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_conv       │ (None, 56, 56, 64)     │         36,928 │ conv2_block1_1_relu[0… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_bn         │ (None, 56, 56, 64)     │            256 │ conv2_block1_2_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_relu       │ (None, 56, 56, 64)     │              0 │ conv2_block1_2_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_0_conv       │ (None, 56, 56, 256)    │         16,640 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_3_conv       │ (None, 56, 56, 256)    │         16,640 │ conv2_block1_2_relu[0… │
│ (Conv2D)             

 Total params: 23,591,810 (90.00 MB)

 Trainable params: 4,098 (16.01 KB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [ ]:
sgd = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(optimizer=sgd, loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
cb_early_stopper = EarlyStopping(monitor = 'val_loss', patience = EARLY_STOP_PATIENCE)
cb_checkpointer = ModelCheckpoint(filepath = 'best_model.keras', monitor = 'val_loss', save_best_only = True, mode = 'auto')

In [ ]:
steps_per_epoch = train_generator.samples // BATCH_SIZE_TRAINING
validation_steps = validation_generator.samples // BATCH_SIZE_VALIDATION

In [ ]:
fit_history = model.fit(
    train_generator,
    steps_per_epoch= steps_per_epoch,
    epochs = NUM_EPOCHS,
    validation_data = validation_generator,
    validation_steps= validation_steps,
    callbacks = [cb_checkpointer, cb_early_stopper]
)

Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
/usr/local/lib/python3.11/dist-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor_6']
Received: inputs=Tensor(shape=(None, 224, 224, 3))
  warnings.warn(msg)


1/1 ━━━━━━━━━━━━━━━━━━━━ 17s 17s/step - accuracy: 0.4444 - loss: 0.7774 - val_accuracy: 0.5000 - val_loss: 3.4009
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step - accuracy: 0.5556 - loss: 1.9691 - val_accuracy: 0.5500 - val_loss: 1.2205
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step - accuracy: 0.8889 - loss: 0.1732 - val_accuracy: 0.7500 - val_loss: 1.0092
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step - accuracy: 1.0000 - loss: 0.0087 - val_accuracy: 0.7500 - val_loss: 1.0480
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 8s 8s/step - accuracy: 1.0000 - loss: 0.0034 - val_accuracy: 0.7500 - val_loss: 1.0982
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 8s 8s/step - accuracy: 1.0000 - loss: 0.0015 - val_accuracy: 0.7500 - val_loss: 1.1511


In [ ]:
# Evaluate the Model on Test Data
test_loss, test_accuracy = model.evaluate(validation_generator)
print(f'Test accuracy: {test_accuracy * 100:.2f}%')

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 0.7500 - loss: 1.1511
Test accuracy: 75.00%


In [ ]:
validation_generator.reset()  # Resetting the generator for new predictions
predictions = model.predict(validation_generator, steps = validation_steps, verbose=1)
predicted_class_indices = np.argmax(predictions, axis=1)

# True class indices from the validation generator
true_class_indices = validation_generator.classes

/usr/local/lib/python3.11/dist-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor_6']
Received: inputs=Tensor(shape=(20, 224, 224, 3))
  warnings.warn(msg)


1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step


In [ ]:
true_class_indices = validation_generator.classes

class_labels = list(validation_generator.class_indices.keys())
predicted_labels = [class_labels[i] for i in predicted_class_indices]
true_labels = [class_labels[i] for i in true_class_indices]

compare_df1 = pd.DataFrame({
    'Filename': validation_generator.filenames,
    'True Class': true_labels,
    'Predicted Class': predicted_labels
})

print(compare_df1)

                        Filename True Class Predicted Class
0   no rust/001_kwmsqgk2.yfv.jpg    no rust         no rust
1   no rust/002_2bvturws.zss.jpg    no rust         no rust
2   no rust/002_dr3selb3.a1h.jpg    no rust         no rust
3   no rust/002_mjcgc4b1.ru0.jpg    no rust            rust
4   no rust/002_yoamnba2.wdq.jpg    no rust            rust
5   no rust/002_zar1fupp.kf2.jpg    no rust         no rust
6   no rust/003_zomak3wq.43c.jpg    no rust            rust
7   no rust/006_2vq3zs35.40h.jpg    no rust            rust
8   no rust/006_esktmdp0.le4.jpg    no rust         no rust
9   no rust/006_pwvc1uiv.zhi.jpg    no rust         no rust
10     rust/002_cql5orz2.p0y.jpg       rust         no rust
11     rust/002_q40jac40.sd4.jpg       rust            rust
12     rust/003_glbjresp.01k.jpg       rust            rust
13     rust/004_1xgwoahx.lqq.jpg       rust         no rust
14     rust/004_i1dmciub.oop.jpg       rust         no rust
15     rust/005_txq3nygx.w0v.jpg       r

In [ ]:
# Calculate final accuracy
correct_predictions = np.sum(predicted_class_indices == true_class_indices)
final_accuracy = correct_predictions / len(true_class_indices)

print(f'Final Overall Accuracy: {final_accuracy * 100:.2f}%')

Final Overall Accuracy: 45.00%


In [ ]:
resnet_dir = 'resent50_test'

os.makedirs(resnet_dir, exist_ok=True) # Resnet50 test directory

In [ ]:
# Save images with predicted labels
for i, filename in enumerate(validation_generator.filenames):
    img_path = os.path.join(test_dir, filename)
    imgBGR = cv2.imread(img_path)  # Read image in BGR format
    imgRGB = cv2.cvtColor(imgBGR, cv2.COLOR_BGR2RGB)  # Convert to RGB format

    # Get the predicted class label
    predicted_class = predicted_labels[i]

    # Add predicted class to the image
    imgRGB = cv2.putText(
        imgRGB, f"True: {true_label} | Predicted: {predicted_label}", (10, 30),
        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2, cv2.LINE_AA
    )

    # Save the output image
    resnet_img_path = os.path.join(resnet_dir, f"test_image_{i+1}.jpg")
    cv2.imwrite(resnet_img_path, cv2.cvtColor(imgRGB, cv2.COLOR_RGB2BGR))

**TASK 2: Develop Mask RCNN for detecting log**

In [ ]:
%pip install labelme2coco

In [ ]:
!unzip log-labelled.zip

In [ ]:
log_data_dir = '/content/log-labelled'

test_log_dir = 'Task2TestImg'

import os
os.makedirs(test_log_dir, exist_ok=True)

In [ ]:
# Select only .png files from the dataset directory
files = [file for file in os.listdir(log_data_dir) if file.endswith('.png')]

# Select 10 random .png files
selected_files = random.sample(files, 10)

# Move the selected files to the test directory
for file in selected_files:
    shutil.move(os.path.join(log_data_dir, file), os.path.join(test_log_dir, file))